In [1]:
pip install lightgbm optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 14.5 MB/s eta 0:00:00


In [2]:
import pandas as pd

df = pd.read_csv("train_features.csv")

target = "failure_24h"
X = df.drop(columns=[target, "RUL"])
y = df[target]

from sklearn.model_selection import train_test_split

units = df["unit"].unique()

train_units, val_units = train_test_split(
    units,
    test_size=0.2,
    random_state=42
)

train_idx = df["unit"].isin(train_units)
val_idx   = df["unit"].isin(val_units)

X_train, X_val = X.loc[train_idx], X.loc[val_idx]
y_train, y_val = y.loc[train_idx], y.loc[val_idx]

In [3]:
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

lgbm_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("model", LGBMClassifier(
        n_estimators=500,
        learning_rate=0.05,
        num_leaves=31,
        class_weight="balanced",
        random_state=42,
        n_jobs=-1
    ))
])

lgbm_pipe.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['sensor_1_roll_std_1' 'sensor_2_roll_std_1' 'sensor_3_roll_std_1'
 'sensor_4_roll_std_1' 'sensor_5_roll_std_1' 'sensor_6_roll_std_1'
 'sensor_7_roll_std_1' 'sensor_8_roll_std_1' 'sensor_9_roll_std_1'
 'sensor_10_roll_std_1' 'sensor_11_roll_std_1' 'sensor_12_roll_std_1'
 'sensor_13_roll_std_1' 'sensor_14_roll_std_1' 'sensor_15_roll_std_1'
 'sensor_16_roll_std_1' 'sensor_17_roll_std_1' 'sensor_18_roll_std_1'
 'sensor_19_roll_std_1' 'sensor_20_roll_std_1' 'sensor_21_roll_std_1']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


[LightGBM] [Info] Number of positive: 25, number of negative: 391
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.044095 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12938
[LightGBM] [Info] Number of data points in the train set: 416, number of used features: 120
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('model',
                 LGBMClassifier(class_weight='balanced', learning_rate=0.05,
                                n_estimators=500, n_jobs=-1,
                                random_state=42))])

In [4]:
from sklearn.metrics import average_precision_score, precision_score, recall_score

y_probs = lgbm_pipe.predict_proba(X_val)[:, 1]

prauc = average_precision_score(y_val, y_probs)

y_pred = (y_probs >= 0.5).astype(int)

precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)

prauc, precision, recall

/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['sensor_1_roll_std_1' 'sensor_2_roll_std_1' 'sensor_3_roll_std_1'
 'sensor_4_roll_std_1' 'sensor_5_roll_std_1' 'sensor_6_roll_std_1'
 'sensor_7_roll_std_1' 'sensor_8_roll_std_1' 'sensor_9_roll_std_1'
 'sensor_10_roll_std_1' 'sensor_11_roll_std_1' 'sensor_12_roll_std_1'
 'sensor_13_roll_std_1' 'sensor_14_roll_std_1' 'sensor_15_roll_std_1'
 'sensor_16_roll_std_1' 'sensor_17_roll_std_1' 'sensor_18_roll_std_1'
 'sensor_19_roll_std_1' 'sensor_20_roll_std_1' 'sensor_21_roll_std_1']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


(np.float64(1.0), 1.0, 0.64)

In [5]:
import optuna
from sklearn.metrics import average_precision_score

def objective(trial):

    params = {
        "n_estimators": trial.suggest_int("n_estimators", 300, 800),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "num_leaves": trial.suggest_int("num_leaves", 16, 128),
        "max_depth": trial.suggest_int("max_depth", 4, 16),
        "min_child_samples": trial.suggest_int("min_child_samples", 20, 100),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
    }

    model = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("model", LGBMClassifier(
            **params,
            class_weight="balanced",
            random_state=42,
            n_jobs=-1
        ))
    ])

    model.fit(X_train, y_train)

    y_probs = model.predict_proba(X_val)[:, 1]

    prauc = average_precision_score(y_val, y_probs)

    return prauc


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=25)

study.best_value, study.best_params

[I 2026-01-31 10:23:32,007] A new study created in memory with name: no-name-6af645d6-4411-446f-9e78-55c778686212
/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['sensor_1_roll_std_1' 'sensor_2_roll_std_1' 'sensor_3_roll_std_1'
 'sensor_4_roll_std_1' 'sensor_5_roll_std_1' 'sensor_6_roll_std_1'
 'sensor_7_roll_std_1' 'sensor_8_roll_std_1' 'sensor_9_roll_std_1'
 'sensor_10_roll_std_1' 'sensor_11_roll_std_1' 'sensor_12_roll_std_1'
 'sensor_13_roll_std_1' 'sensor_14_roll_std_1' 'sensor_15_roll_std_1'
 'sensor_16_roll_std_1' 'sensor_17_roll_std_1' 'sensor_18_roll_std_1'
 'sensor_19_roll_std_1' 'sensor_20_roll_std_1' 'sensor_21_roll_std_1']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


[LightGBM] [Info] Number of positive: 25, number of negative: 391
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001106 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12938
[LightGBM] [Info] Number of data points in the train set: 416, number of used features: 120
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['sensor_1_roll_std_1' 'sensor_2_roll_std_1' 'sensor_3_roll_std_1'
 'sensor_4_roll_std_1' 'sensor_5_roll_std_1' 'sensor_6_roll_std_1'
 'sensor_7_roll_std_1' 'sensor_8_roll_std_1' 'sensor_9_roll_std_1'
 'sensor_10_roll_std_1' 'sensor_11_roll_std_1' 'sensor_12_roll_std_1'
 'sensor_13_roll_std_1' 'sensor_14_roll_std_1' 'sensor_15_roll_std_1'
 'sensor_16_roll_std_1' 'sensor_17_roll_std_1' 'sensor_18_roll_std_1'
 'sensor_19_roll_std_1' 'sensor_20_roll_std_1' 'sensor_21_roll_std_1']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2026-01-31 10:23:32,405] Trial 0 finished with value: 0.9220666252918824 and parameters: {'n_estim

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped t

/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['sensor_1_roll_std_1' 'sensor_2_roll_std_1' 'sensor_3_roll_std_1'
 'sensor_4_roll_std_1' 'sensor_5_roll_std_1' 'sensor_6_roll_std_1'
 'sensor_7_roll_std_1' 'sensor_8_roll_std_1' 'sensor_9_roll_std_1'
 'sensor_10_roll_std_1' 'sensor_11_roll_std_1' 'sensor_12_roll_std_1'
 'sensor_13_roll_std_1' 'sensor_14_roll_std_1' 'sensor_15_roll_std_1'
 'sensor_16_roll_std_1' 'sensor_17_roll_std_1' 'sensor_18_roll_std_1'
 'sensor_19_roll_std_1' 'sensor_20_roll_std_1' 'sensor_21_roll_std_1']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


...

/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['sensor_1_roll_std_1' 'sensor_2_roll_std_1' 'sensor_3_roll_std_1'
 'sensor_4_roll_std_1' 'sensor_5_roll_std_1' 'sensor_6_roll_std_1'
 'sensor_7_roll_std_1' 'sensor_8_roll_std_1' 'sensor_9_roll_std_1'
 'sensor_10_roll_std_1' 'sensor_11_roll_std_1' 'sensor_12_roll_std_1'
 'sensor_13_roll_std_1' 'sensor_14_roll_std_1' 'sensor_15_roll_std_1'
 'sensor_16_roll_std_1' 'sensor_17_roll_std_1' 'sensor_18_roll_std_1'
 'sensor_19_roll_std_1' 'sensor_20_roll_std_1' 'sensor_21_roll_std_1']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


[LightGBM] [Info] Number of positive: 25, number of negative: 391
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.035450 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12938
[LightGBM] [Info] Number of data points in the train set: 416, number of used features: 120
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['sensor_1_roll_std_1' 'sensor_2_roll_std_1' 'sensor_3_roll_std_1'
 'sensor_4_roll_std_1' 'sensor_5_roll_std_1' 'sensor_6_roll_std_1'
 'sensor_7_roll_std_1' 'sensor_8_roll_std_1' 'sensor_9_roll_std_1'
 'sensor_10_roll_std_1' 'sensor_11_roll_std_1' 'sensor_12_roll_std_1'
 'sensor_13_roll_std_1' 'sensor_14_roll_std_1' 'sensor_15_roll_std_1'
 'sensor_16_roll_std_1' 'sensor_17_roll_std_1' 'sensor_18_roll_std_1'
 'sensor_19_roll_std_1' 'sensor_20_roll_std_1' 'sensor_21_roll_std_1']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2026-01-31 10:23:52,429] Trial 24 finished with value: 0.9383366302899119 and parameters: {'n_esti

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

(0.97997335997336,
 {'n_estimators': 645,
  'learning_rate': 0.040157865064727,
  'num_leaves': 57,
  'max_depth': 9,
  'min_child_samples': 70,
  'subsample': 0.8111116983451957,
  'colsample_bytree': 0.7652835045788573})

In [6]:
best_params = study.best_params

final_lgbm = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("model", LGBMClassifier(
        **best_params,
        class_weight="balanced",
        random_state=42,
        n_jobs=-1
    ))
])

final_lgbm.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['sensor_1_roll_std_1' 'sensor_2_roll_std_1' 'sensor_3_roll_std_1'
 'sensor_4_roll_std_1' 'sensor_5_roll_std_1' 'sensor_6_roll_std_1'
 'sensor_7_roll_std_1' 'sensor_8_roll_std_1' 'sensor_9_roll_std_1'
 'sensor_10_roll_std_1' 'sensor_11_roll_std_1' 'sensor_12_roll_std_1'
 'sensor_13_roll_std_1' 'sensor_14_roll_std_1' 'sensor_15_roll_std_1'
 'sensor_16_roll_std_1' 'sensor_17_roll_std_1' 'sensor_18_roll_std_1'
 'sensor_19_roll_std_1' 'sensor_20_roll_std_1' 'sensor_21_roll_std_1']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


[LightGBM] [Info] Number of positive: 25, number of negative: 391
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001107 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12938
[LightGBM] [Info] Number of data points in the train set: 416, number of used features: 120
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('model',
                 LGBMClassifier(class_weight='balanced',
                                colsample_bytree=0.7652835045788573,
                                learning_rate=0.040157865064727, max_depth=9,
                                min_child_samples=70, n_estimators=645,
                                n_jobs=-1, num_leaves=57, random_state=42,
                                subsample=0.8111116983451957))])

In [7]:
import numpy as np

y_probs = final_lgbm.predict_proba(X_val)[:, 1]

thresholds = np.linspace(0.1, 0.9, 50)

results = []

from sklearn.metrics import precision_score, recall_score

for t in thresholds:
    y_pred = (y_probs >= t).astype(int)
    results.append({
        "threshold": t,
        "precision": precision_score(y_val, y_pred),
        "recall": recall_score(y_val, y_pred),
    })

results_df = pd.DataFrame(results)
results_df.sort_values(by="precision", ascending=False).head(10)

/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['sensor_1_roll_std_1' 'sensor_2_roll_std_1' 'sensor_3_roll_std_1'
 'sensor_4_roll_std_1' 'sensor_5_roll_std_1' 'sensor_6_roll_std_1'
 'sensor_7_roll_std_1' 'sensor_8_roll_std_1' 'sensor_9_roll_std_1'
 'sensor_10_roll_std_1' 'sensor_11_roll_std_1' 'sensor_12_roll_std_1'
 'sensor_13_roll_std_1' 'sensor_14_roll_std_1' 'sensor_15_roll_std_1'
 'sensor_16_roll_std_1' 'sensor_17_roll_std_1' 'sensor_18_roll_std_1'
 'sensor_19_roll_std_1' 'sensor_20_roll_std_1' 'sensor_21_roll_std_1']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


,threshold,precision,recall
0,0.100000,1.0,0.64
1,0.116327,1.0,0.64
2,0.132653,1.0,0.64
3,0.148980,1.0,0.64
4,0.165306,1.0,0.64
5,0.181633,1.0,0.64
6,0.197959,1.0,0.64
7,0.214286,1.0,0.64
8,0.230612,1.0,0.64
9,0.246939,1.0,0.64


In [8]:
import joblib

joblib.dump(final_lgbm, "factoryguard_final_model.joblib")


['factoryguard_final_model.joblib']